In [1]:
import os

wd = os.path.normpath(os.getcwd() + '/..')
os.chdir(wd)
os.getcwd()

'/home/toc/TOC/Choi/MultiAgentTrajectoryImputation-ICLR2024'

In [3]:
%load_ext autoreload
%autoreload 2

import glob
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
from matplotlib import animation

from tqdm import tqdm

from nba_helper import NBADataHelper, NBADataAnimator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Processing NBA data(Single match)

In [ ]:
data_path = f"./data/2016.NBA.Raw.SportVU.Game.Logs"
matches = glob.glob(data_path + "/*.json")
print(f"The number of NBA matches : {len(matches)}")

In [ ]:
match = matches[0]
match_id = match.split("/")[-1][:-5]
print(f"Match Id: {match_id}")

In [ ]:
match_df = pd.read_json(matches[0]) # Select sample match
helper = NBADataHelper(match_df)

In [ ]:
smoothing = True
remove_outliers = False

helper.reconstruct_df()
helper.downsample_to_10fps()
helper.split_into_episodes()
helper.calc_running_features(remove_outliers=remove_outliers, smoothing=smoothing)
helper.traces

In [ ]:
# helper.traces.to_csv(f"data/nba_traces/match{match_id}.csv", index=False)

### Processing NBA data(Entire matches)

In [ ]:
data_path = f"./data/2016.NBA.Raw.SportVU.Game.Logs"
matches = glob.glob(data_path + "/*.json")
matches.sort()
matches = matches[:100]
for match_idx in range(len(matches)):
    match = matches[match_idx]
    match_id = match.split("/")[-1][:-5]

    print(f"Match Id: {match_id}")

    match_df = pd.read_json(matches[match_idx])
    helper = NBADataHelper(match_df)

    helper.reconstruct_df()
    helper.split_into_episodes()
    helper.downsample_to_10fps()
    helper.calc_running_features(remove_outliers=False, smoothing=True)
    
    helper.traces.to_csv(f"data/nba_traces/match{match_id}.csv", index=False)

### Visualization for NBA Data

##### Animating Trajectories

In [ ]:
quarter = 4
traces = helper.traces[helper.traces["quarter"] == quarter]
# episode = 1
# traces = helper.traces[helper.traces["episode"] == episode]

In [ ]:
animator = NBADataAnimator(traces, show_frames=False, show_episodes=True)

anim = animator.run()

path = f"animations/match_id_{match_id}_quarter{quarter}_latest.mp4"
writer = animation.FFMpegWriter(fps=10)
anim.save(path, writer=writer)

##### Animating Feature Plots


In [ ]:
anim = NBADataAnimator.plot_speeds_and_accels(helper.traces, helper.player_cols)
writer = animation.FFMpegWriter(fps=5)

path = f"animations/feature_plots/latest.mp4"
# path = f"animations/feature_plots/NBA_match{match_id}_quarter{quarter}.mp4"
    
anim.save(path, writer=writer)

### Debug 

##### Test vel_pos

In [ ]:
traces["episode"].unique()

In [ ]:
traces = helper.traces
traces

In [ ]:
# feature_types = ["_x", "_y", "_vx", "_vy", "_ax", "_ay"]
feature_types = ["_x", "_y", "_vx", "_vy", "_speed", "_accel", "_ax", "_ay"]
data_cols = [f"player{p}{t}" for p in range(10) for t in feature_types]
len(data_cols)

In [ ]:
traces = traces[["episode"] + data_cols]
episode_traces = traces[traces["episode"] == 1]
episode_traces

In [ ]:
episode_traces = episode_traces[data_cols].values
episode_traces.shape

In [ ]:
pos_x = torch.tensor(episode_traces[:, 0::8, None])
pos_y = torch.tensor(episode_traces[:, 1::8, None])
pos_xy = torch.cat([pos_x, pos_y], dim=-1)

vel_x = torch.tensor(episode_traces[:, 2::8, None])
vel_y = torch.tensor(episode_traces[:, 3::8, None])
vel_xy = torch.cat([vel_x, vel_y], dim=-1)

speed = torch.tensor(episode_traces[:, 4::8])
accel = torch.tensor(episode_traces[:, 5::8])

print(pos_xy.shape, vel_xy.shape)

In [ ]:
vel_pos = torch.zeros((pos_xy.shape[0], 10, 2))
vel_pos[0] = pos_xy[0]
vel_pos[1:] += vel_xy[:-1] * 0.1
vel_pos.cumsum_(dim=0)

In [ ]:
pos_dist = torch.norm((pos_xy[1:] - vel_pos[1:]), dim=-1)
pos_dist_np = np.array(pos_dist)
pos_dist_np.shape

In [ ]:
cols = [f"player{p}" for p in range(10)]
pos_dist_df = pd.DataFrame(pos_dist_np, columns=cols)

import random
colors = [f'#{random.randint(0, 0xFFFFFF):06x}' for _ in range(10)]

pos_dist_df.plot(figsize=(12, 6), color=colors, markersize=1, legend=True)

plt.title("Positional Distance for 10 Players")
plt.xlabel("Time")
plt.ylabel("Distance")

plt.tight_layout()
plt.show()

##### Check anomaly frames in episodes

In [ ]:
traces = helper.traces
traces

In [ ]:
episodes = traces["episode"].unique()
episodes

In [ ]:
for e in episodes:
    episode_traces = traces[traces["episode"] == e]
    episode_xy_traces = episode_traces[helper.player_xy_cols]

    frame_diff = np.diff(episode_xy_traces, axis=0, prepend=episode_xy_traces[:1])
    frame_diff_dist = np.sqrt(frame_diff[:, 0::2] ** 2 + frame_diff[:, 1::2] ** 2)
    
    if (frame_diff_dist > 3.0).sum() > 0:
        print(f"Anomaly frmaes in episode_{e}")